In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from google.cloud import bigquery

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t

client = bigquery.Client()

#########################################################
#                Graphs for section 6                   #
#########################################################

# Figure 4:  International vs Domestic Online Ratio     

In [ ]:
# Total spend in foreign countries by UK cardholders
total_spend_abroad = '''SELECT time_period_value, sum(spend) as total_spend_abroad
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Month'
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'All'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
  AND destination_country != 'UNITED KINGDOM' 
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
spend_abroad = bq.read_bq_table_sql(client, total_spend_abroad)
#spend_abroad.head()

In [ ]:
# Total online spend in foreign countries by UK cardholders
total_online_spend_abroad = '''SELECT time_period_value, sum(spend) as total_online_spend_abroad
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Month' 
  AND mcg = 'All'
  AND mcc = 'All'
  AND merchant_channel = 'Online'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
  AND destination_country != 'UNITED KINGDOM' 
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
online_spend_abroad = bq.read_bq_table_sql(client, total_online_spend_abroad)
#online_spend_abroad.head()

In [ ]:
# Total spend in the UK by UK cardholders
total_spend_domestic = '''SELECT time_period_value, sum(spend) as total_spend_domestic
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Month' 
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'All'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All' 
  AND destination_country = 'UNITED KINGDOM'
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
spend_domestic = bq.read_bq_table_sql(client, total_spend_domestic)
#spend_domestic.head()

In [ ]:
# Total online spend in UK by UK cardholders
total_online_spend_domestic = '''SELECT time_period_value, sum(spend) as total_online_spend_domestic
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Month' 
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'Online'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
  AND destination_country = 'UNITED KINGDOM' 
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
online_spend_domestic = bq.read_bq_table_sql(client, total_online_spend_domestic)
#online_spend_domestic.head()

In [ ]:
# Merging abroad figures together 
merged_abroad_spending = pd.merge(spend_abroad, online_spend_abroad, on='time_period_value')

# Merging domestic figures together
merged_dom_spending = pd.merge(spend_domestic, online_spend_domestic, on='time_period_value')

merged_dom_spending

In [ ]:
# Calculating the online % ratio for each

merged_abroad_spending["Abroad Online %"] = (merged_abroad_spending['total_online_spend_abroad'] /  merged_abroad_spending['total_spend_abroad']) * 100
merged_dom_spending["Domestic Online %"] = (merged_dom_spending['total_online_spend_domestic'] /  merged_dom_spending['total_spend_domestic']) * 100

merged_abroad_spending

In [ ]:
# Gathering online ratios into one table
int_vs_dom_ratios = merged_abroad_spending
int_vs_dom_ratios = pd.merge(int_vs_dom_ratios, merged_dom_spending, on='time_period_value')

int_vs_dom_ratios = int_vs_dom_ratios.drop(columns=["total_spend_abroad", "total_online_spend_abroad", "total_spend_domestic", "total_online_spend_domestic"])
#int_vs_dom_ratios.to_csv("Figure 4: International vs Domestic Online Ratio.csv")

In [ ]:
# Line chart for the Figure 4
fig4 = px.line(
        int_vs_dom_ratios,
        x="time_period_value",
        y=["Abroad Online %", "Domestic Online %"],
    )
fig4

# Figure 5:  Domestic Online Spend at Retail MCCs vs RSI Internet Sales %

In [ ]:
# Domestic spend at selected MCCs that relate to SIC 47 (Retail Sector)
total_by_mcc = '''SELECT time_period_value, sum(spend) as total_spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Month'
  AND mcg != 'All' 
  AND mcc in ("ANTIQUE REPRODUCTION STORES",
"ANTIQUE SHOPS",
"ART DEALERS & GALLERIES",
"ARTIST/CRAFT SHOPS",
"AUTOMATED FUEL DISPENSERS",
"BAKERIES",
"BICYCLE SHOPS/SALES/SERVICE",
"BOOK STORES",
"CAMERA & PHOTO SUPPLY STORES",
"CANDY/NUT/CONFECTION STORES",
"CATALOG MERCHANT",
"CHILDREN/INFANTS WEAR STORES",
"COMBINATION CATALOG & RETAIL",
"COMPUTER SOFTWARE STORES",
"CONTINUITY/SUBSCRIPTION MERCHT",
"COSMETIC STORES",
"DAIRY PRODUCT STORES",
"DEPARTMENT STORES",
"DIGITAL GOODS APP(EXCL GAMES)",
"DIGITAL GOODS BOOKSMOVIEMUSIC",
"DIGITAL GOODS GAMES",
"DIRECT SELL/DOOR-TO-DOOR",
"DISCOUNT STORES",
"DRAPERY & UPHOLSTERY STORES",
"DUTY FREE STORES",
"ELEC RAZOR STORES/SALE/SERV",
"ELECTRIC VEHICLE CHARGING",
"ELECTRONICS STORES",
"FABRIC STORES",
"FAMILY CLOTHING STORES",
"FILMS/VIDEO PRODUCTION/DIST",
"FIREPLACES & ACCESSORIES",
"FLOOR COVERING STORES",
"FLORISTS",
"FREEZER/MEAT LOCKERS",
"FUEL DEALERS",
"FURNITURE/EQUIP STORES",
"FURRIERS AND FUR SHOPS",
"GIFT, CARD, NOVELTY STORES",
"GLASS/PAINT/WALLPAPER STORES",
"GLASSWARE/CRYSTAL STORES",
"GROCERY STORES/SUPERMARKETS",
"HARDWARE STORES",
"HOBBY, TOY & GAME STORES",
"HOME SUPPLY WAREHOUSE STORES",
"HOUSEHOLD APPLIANCE STORES",
"INBOUND TELEMARKETING MERCHANT",
"JEWELRY STORES",
"LARGE DIGITAL GOODS MERCHANT",
"LUGGAGE/LEATHER STORES",
"MEN/BOYS CLOTHING/ACC STORES",
"MENS/WOMENS CLOTHING STORES",
"MISC APPAREL/ACCESS STORES",
"MISC FOOD STORES - DEFAULT",
"MISC GENERAL MERCHANDISE",
"MISC HOME FURNISHING SPECIALTY",
"MISC SPECIALTY RETAIL",
"MUSIC STORES/PIANOS",
"NEWS DEALERS/NEWSSTANDS",
"NURSERIES, LAWN/GARDEN SUPPLY",
"ONLINE MARKETPLACES",
"OTHER DIRECT MARKETERS",
"OUTBOUND TELEMARKETING MERCHNT",
"PET STORES/FOOD & SUPPLY",
"RECORD STORES",
"RELIGIOUS GOODS STORES",
"SERVICE STATIONS",
"SHOE STORES",
"SPORTING GOODS STORES",
"SPORTS/RIDING APPAREL STORES",
"STAMP & COIN STORES",
"STATIONERY STORES",
"TENT AND AWNING SHOPS",
"USED MERCHANDISE STORES",
"VARIETY STORES",
"VIDEO AMUSEMENT GAME SUPPLY",
"WIG AND TOUPEE STORES",
"WOMENS ACCESS/SPECIALTY",
"WOMENS READY TO WEAR STORES")
  AND merchant_channel = 'All'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
  AND destination_country = 'UNITED KINGDOM'
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
spend_by_retail_mcc = bq.read_bq_table_sql(client, total_by_mcc)
spend_by_retail_mcc

In [ ]:
# Domestic spend at selected MCCs that relate to SIC 47 (Retail Sector)
online_by_mcc = '''SELECT time_period_value, sum(spend) as online_spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Month'
  AND mcg != 'All' 
  AND mcc in ("ANTIQUE REPRODUCTION STORES",
"ANTIQUE SHOPS",
"ART DEALERS & GALLERIES",
"ARTIST/CRAFT SHOPS",
"AUTOMATED FUEL DISPENSERS",
"BAKERIES",
"BICYCLE SHOPS/SALES/SERVICE",
"BOOK STORES",
"CAMERA & PHOTO SUPPLY STORES",
"CANDY/NUT/CONFECTION STORES",
"CATALOG MERCHANT",
"CHILDREN/INFANTS WEAR STORES",
"COMBINATION CATALOG & RETAIL",
"COMPUTER SOFTWARE STORES",
"CONTINUITY/SUBSCRIPTION MERCHT",
"COSMETIC STORES",
"DAIRY PRODUCT STORES",
"DEPARTMENT STORES",
"DIGITAL GOODS APP(EXCL GAMES)",
"DIGITAL GOODS BOOKSMOVIEMUSIC",
"DIGITAL GOODS GAMES",
"DIRECT SELL/DOOR-TO-DOOR",
"DISCOUNT STORES",
"DRAPERY & UPHOLSTERY STORES",
"DUTY FREE STORES",
"ELEC RAZOR STORES/SALE/SERV",
"ELECTRIC VEHICLE CHARGING",
"ELECTRONICS STORES",
"FABRIC STORES",
"FAMILY CLOTHING STORES",
"FILMS/VIDEO PRODUCTION/DIST",
"FIREPLACES & ACCESSORIES",
"FLOOR COVERING STORES",
"FLORISTS",
"FREEZER/MEAT LOCKERS",
"FUEL DEALERS",
"FURNITURE/EQUIP STORES",
"FURRIERS AND FUR SHOPS",
"GIFT, CARD, NOVELTY STORES",
"GLASS/PAINT/WALLPAPER STORES",
"GLASSWARE/CRYSTAL STORES",
"GROCERY STORES/SUPERMARKETS",
"HARDWARE STORES",
"HOBBY, TOY & GAME STORES",
"HOME SUPPLY WAREHOUSE STORES",
"HOUSEHOLD APPLIANCE STORES",
"INBOUND TELEMARKETING MERCHANT",
"JEWELRY STORES",
"LARGE DIGITAL GOODS MERCHANT",
"LUGGAGE/LEATHER STORES",
"MEN/BOYS CLOTHING/ACC STORES",
"MENS/WOMENS CLOTHING STORES",
"MISC APPAREL/ACCESS STORES",
"MISC FOOD STORES - DEFAULT",
"MISC GENERAL MERCHANDISE",
"MISC HOME FURNISHING SPECIALTY",
"MISC SPECIALTY RETAIL",
"MUSIC STORES/PIANOS",
"NEWS DEALERS/NEWSSTANDS",
"NURSERIES, LAWN/GARDEN SUPPLY",
"ONLINE MARKETPLACES",
"OTHER DIRECT MARKETERS",
"OUTBOUND TELEMARKETING MERCHNT",
"PET STORES/FOOD & SUPPLY",
"RECORD STORES",
"RELIGIOUS GOODS STORES",
"SERVICE STATIONS",
"SHOE STORES",
"SPORTING GOODS STORES",
"SPORTS/RIDING APPAREL STORES",
"STAMP & COIN STORES",
"STATIONERY STORES",
"TENT AND AWNING SHOPS",
"USED MERCHANDISE STORES",
"VARIETY STORES",
"VIDEO AMUSEMENT GAME SUPPLY",
"WIG AND TOUPEE STORES",
"WOMENS ACCESS/SPECIALTY",
"WOMENS READY TO WEAR STORES")
  AND merchant_channel = 'Online'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
  AND destination_country = 'UNITED KINGDOM'
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
online_spend_by_retail_mcc = bq.read_bq_table_sql(client, online_by_mcc)
online_spend_by_retail_mcc

In [ ]:
# Calculating the ratio of domestic spend that is online for our selected MCCs
retail_online_ratio = pd.merge(spend_by_retail_mcc, online_spend_by_retail_mcc, on="time_period_value")
retail_online_ratio["Online %"] = (retail_online_ratio["online_spend"] / retail_online_ratio["total_spend"]) * 100

retail_online_ratio = retail_online_ratio.drop(columns=["total_spend", "online_spend"])
retail_online_ratio

In [ ]:
# Getting the ratio of sales that are over the internet published by RSI
RSI_online_ratio = pd.read_csv("/home/jupyter/ft_digital_trade/data/raw/RSI Internet Sales.csv")

# Combining RSI internet sales and spend at 'retail' MCCs online %
RSI_online_ratio["Retail MCCs Online %"] = retail_online_ratio["Online %"]
RSI_online_ratio["time_period_value"] = RSI_online_ratio["time_period_value"].astype(str)

RSI_vs_retail_mccs = RSI_online_ratio
#RSI_vs_retail_mccs.to_csv("Figure 5: RSI Internet Sales % vs Retail MCCs Online %.csv")

In [ ]:
# Line chart for the Figure 5
fig5 = px.line(
        RSI_vs_retail_mccs,
        x="time_period_value",
        y=["RSI Internet Sales %", "Retail MCCs Online %"],
    )
fig5

# Table 1: International & Online Spend as % of Total

In [ ]:
# Total spend by UK cardholders
total_spend = '''SELECT time_period_value, sum(spend) as total_spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Month' 
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'All'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All' 
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
spend_total = bq.read_bq_table_sql(client, total_spend)

spend_total['Year'] = spend_total['time_period_value'].str[:4].astype(int)

yearly_total_spend = spend_total.groupby(['Year'])['total_spend'].sum().reset_index()
#yearly_total_spend

In [ ]:
online_spend_abroad['Year'] = online_spend_abroad['time_period_value'].str[:4].astype(int)
yearly_abr_online_spend = online_spend_abroad.groupby(['Year'])['total_online_spend_abroad'].sum().reset_index()

total_vs_intonline = pd.merge(yearly_total_spend, yearly_abr_online_spend, on="Year")
total_vs_intonline["International + Online Spend Ratio"] = (total_vs_intonline["total_online_spend_abroad"] / total_vs_intonline["total_spend"] ) * 100

total_vs_intonline = total_vs_intonline.drop(columns=["total_spend", "total_online_spend_abroad"])
total_vs_intonline

In [ ]:
# Table 1:
#total_vs_intonline.to_csv("Table 1: Ratio of spend that is online and abroad, yearly.csv")

#########################################################
#                Analysis for section 6                 #
#########################################################

In [ ]:
# Calculating the averages over whole time series for international and domestic online ratios
int_average = int_vs_dom_ratios["Abroad Online %"].mean()
dom_average = int_vs_dom_ratios["Domestic Online %"].mean()

print(f"Average online ratio for international spend over time series was {int_average}")
print(f"Average online ratio for domestic spend over time series was {dom_average}")

In [ ]:
# Calculating the average ratios for both in 2019 and 2024 to see how the 'gap' between them has fallen
# Making a year column
int_vs_dom_ratios["Year"] = int_vs_dom_ratios['time_period_value'].str[:4]

# Same average spend function as used in Section 4 code
def average_spend(df, column_name, year=2019):
    # Convert year to string for comparison
    year_str = str(year)
    
    # Filter rows where 'time_period_value' starts with the year
    filtered_df = df[df['time_period_value'].astype(str).str.startswith(year_str)]
    
    # Calculate and return the average
    return filtered_df[column_name].mean()

int_average_2019 = average_spend(int_vs_dom_ratios, "Abroad Online %", 2019)
int_average_2024 = average_spend(int_vs_dom_ratios, "Abroad Online %", 2024)
int_average_2025 = average_spend(int_vs_dom_ratios, "Abroad Online %", 2025) # Value used in graph title

dom_average_2019 = average_spend(int_vs_dom_ratios, "Domestic Online %", 2019)
dom_average_2024 = average_spend(int_vs_dom_ratios, "Domestic Online %", 2024)
dom_average_2019

gap_in_2019 = int_average_2019 - dom_average_2019
gap_in_2024 = int_average_2024 - dom_average_2024

print(f"Difference in ratios in 2019 was {gap_in_2019} but only {gap_in_2024} in 2024")
print(f"In 2025, an average of {int_average_2025} of international transations were made online")

In [ ]:
# Peaks in each series
int_peak = int_vs_dom_ratios.nlargest(1, "Abroad Online %")
int_peak = int_vs_dom_ratios.nlargest(1, "Domestic Online %")

print(int_peak)
print(dom_peak)

In [ ]:
# Difference between 2024 and 2019 average values 

int_difference = int_average_2024 - int_average_2019
dom_difference = dom_average_2024 - dom_average_2019

print(f"From 2019 to 2024, the international online ratio has decreased by {int_difference} while the domestic online ratio has increased by {dom_difference}")

In [ ]:
# June 2025 compared with June 2019
starting_month = "201906"
finishing_month = "202506"

def nom_growth_between_months(starting_month, finishing_month, table, column):
    start_rows = table.loc[table['time_period_value'] == starting_month, column]
    end_rows = table.loc[table['time_period_value'] == finishing_month, column]

    if start_rows.empty or end_rows.empty:
        raise ValueError(f"Missing data for: {starting_month if start_rows.empty else finishing_month}")

    start_value = start_rows.values[0]
    end_value = end_rows.values[0]

    difference = end_value - start_value
    return difference

int_june19_to_june25 = nom_growth_between_months(starting_month, finishing_month, int_vs_dom_ratios, "Abroad Online %")
dom_june19_to_june25 = nom_growth_between_months(starting_month, finishing_month, int_vs_dom_ratios, "Domestic Online %")

print(f"From June 2019 to June 2025, the international online ratio has fallen by {int_june19_to_june25} while it has increased by {dom_june19_to_june25} for domestic transactions")

In [ ]:
# Calculating the average growth for each destination in each month from 202201 onwards
### If using monthly values (like this) the result is X.3 but when using quarterly values it is X.2 ###
# Creating temporary copies to avoid SettingWithCopyWarning
temp_df = int_vs_dom_ratios.copy()

# Extract year and month
temp_df['quarter'] = temp_df['time_period_value'].str[-2:].str.replace('Q', '').astype(int)
temp_df['year'] = temp_df['time_period_value'].str[:4].astype(int)
# Filter for 202201 and later
temp_df = temp_df[(temp_df['year'] > 2021)]
int_average_since_2022 = temp_df["Abroad Online %"].mean()


# Recalculate month for grouping
temp_df['month'] = temp_df['time_period_value'].str[-2:]

# Group and calculate average percent_change in each month
average_df_2022 = temp_df.groupby(['month'])[['Abroad Online %']].mean().reset_index()

int_average_since_2022

In [ ]:
# Quarterly queries to look at seasonal (summer) trends in international spending
# Total spend in foreign countries by UK cardholders
quarterly_spend_abroad = '''SELECT time_period_value, sum(spend) as total_spend_abroad
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Quarter'
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'All'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
  AND destination_country != 'UNITED KINGDOM' 
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
quarterly_spend_abroad = bq.read_bq_table_sql(client, quarterly_spend_abroad)
#quarterly_spend_abroad.head()

In [ ]:
# Online spend in foreign countries by UK cardholders
quarterly_spend_online_abroad = '''SELECT time_period_value, sum(spend) as online_spend_abroad
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel`
WHERE time_period = 'Quarter'
  AND mcg = 'All' 
  AND mcc = 'All'
  AND merchant_channel = 'Online'
  AND cardholder_origin = 'UNITED KINGDOM'
  AND cardholder_origin_country = 'All'
  AND destination_country != 'UNITED KINGDOM' 
GROUP BY time_period_value
ORDER BY time_period_value ASC'''
quarterly_online_spend_abroad = bq.read_bq_table_sql(client, quarterly_spend_online_abroad)
#quarterly_online_spend_abroad.head()

In [ ]:
merged_quarterly_spend_abroad = pd.merge(quarterly_spend_abroad, quarterly_online_spend_abroad, on="time_period_value")

merged_quarterly_spend_abroad

In [ ]:
# Calculating change values for online and f2f spending
merged_quarterly_spend_abroad['total_percent_change'] = merged_quarterly_spend_abroad['total_spend_abroad'].pct_change() * 100
merged_quarterly_spend_abroad['online_percent_change'] = merged_quarterly_spend_abroad['online_spend_abroad'].pct_change() * 100
merged_quarterly_spend_abroad["Abroad Online %"] = (merged_quarterly_spend_abroad["online_spend_abroad"] / merged_quarterly_spend_abroad["total_spend_abroad"]) * 100

merged_quarterly_spend_abroad

In [ ]:
# Calculating the average growth for each destination in each quarter from 202201 onwards 

# Creating temporary copies to avoid SettingWithCopyWarning
temp_df = merged_quarterly_spend_abroad.copy()

# Extract year and month
temp_df['quarter'] = temp_df['time_period_value'].str[-2:].str.replace('Q', '').astype(int)
temp_df['year'] = temp_df['time_period_value'].str[:4].astype(int)
# Filter for 2022Q1 and later
temp_df = temp_df[(temp_df['year'] > 2021)]
int_average_since_2022 = temp_df["Abroad Online %"].mean()


# Recalculate month for grouping
temp_df['quarter'] = temp_df['time_period_value'].str[-2:]

# Group and calculate average online ratio in each quarter from 2022 onwards
average_df_2022 = temp_df.groupby(['quarter'])[['Abroad Online %']].mean().reset_index()
average_qoq_growth_online = temp_df.groupby(['quarter'])[['online_percent_change']].mean().reset_index()
average_qoq_growth = temp_df.groupby(['quarter'])[['total_percent_change']].mean().reset_index()

print(f"Average online ratio from Jan 2022 to June 2025 is: {int_average_since_2022}")
print(f"Average % growth QonQ in international spend from 2022 to 2025: {average_qoq_growth}")
print(f"Average % growth QonQ in international online spend from 2022 to 2025: {average_qoq_growth_online}")

In [ ]:
# Analysis for the comparison of the 'retail MCCs' with RSI internet sales
RSI_vs_retail_mccs

In [ ]:
# Calculating growth rates between June 2019 and June 2025
starting_month = "201906"
finishing_month = "202506"

retail_growth = nom_growth_between_months(starting_month, finishing_month, RSI_vs_retail_mccs, "Retail MCCs Online %")
RSI_growth = nom_growth_between_months(starting_month, finishing_month, RSI_vs_retail_mccs, "RSI Internet Sales %")

print(f"Between June 2019 and June 2025, retail MCCs online ratio grew by {retail_growth} while RSI internet sales grew by {RSI_growth}")

In [ ]:
# Average gap between the 2 series

difference = (RSI_vs_retail_mccs["Retail MCCs Online %"].astype(int).mean() - RSI_vs_retail_mccs["RSI Internet Sales %"].astype(int).mean())

print(f"Average difference between retail spend online ratio and RSI internet sales %: {difference}")

In [ ]:
# Needs some more analysis, not sure what yet